In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import np_utils
from keras.constraints import maxnorm
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from numpy import int64

In [8]:
data=pd.read_csv("forestfires.csv")
data

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,...,0,0,0,0,1,0,0,0,0,small
1,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,...,0,0,0,0,0,0,0,1,0,small
2,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,...,0,0,0,0,0,0,0,1,0,small
3,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,...,0,0,0,0,1,0,0,0,0,small
4,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,...,0,0,0,0,1,0,0,0,0,small
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,aug,sun,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,...,0,0,0,0,0,0,0,0,0,large
513,aug,sun,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,...,0,0,0,0,0,0,0,0,0,large
514,aug,sun,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,...,0,0,0,0,0,0,0,0,0,large
515,aug,sat,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,...,0,0,0,0,0,0,0,0,0,small


In [9]:
# Dropping columns which are not required

data = data.drop(['dayfri', 'daymon', 'daysat', 'daysun', 'daythu','daytue', 'daywed', 'monthapr', 'monthaug', 'monthdec', 
                  'monthfeb','monthjan', 'monthjul', 'monthjun', 'monthmar', 'monthmay', 'monthnov','monthoct','monthsep'], 
                 axis = 1)
data

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,size_category
0,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.00,small
1,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.00,small
2,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.00,small
3,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.00,small
4,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.00,small
...,...,...,...,...,...,...,...,...,...,...,...,...
512,aug,sun,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,6.44,large
513,aug,sun,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,54.29,large
514,aug,sun,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,11.16,large
515,aug,sat,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,0.00,small


In [11]:
data.month.replace(('jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec'),(1,2,3,4,5,6,7,8,9,10,11,12), inplace=True)
data.day.replace(('mon','tue','wed','thu','fri','sat','sun'),(1,2,3,4,5,6,7), inplace=True)

In [13]:
# Encoding target variable 'size category'

data.size_category.replace(('small', 'large'), (0, 1), inplace = True)
data.sample(5)

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,size_category
476,7,4,93.5,85.3,395.0,9.9,27.2,28,1.3,0.0,1.76,0
78,8,5,90.1,108.0,529.8,12.5,14.7,66,2.7,0.0,0.00,0
179,8,2,88.8,147.3,614.5,9.0,14.4,66,5.4,0.0,5.23,0
222,3,1,87.6,52.2,103.8,5.0,11.0,46,5.8,0.0,36.85,1
310,9,7,92.4,105.8,758.1,9.9,25.3,27,2.7,0.0,0.00,0


In [14]:
data

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,size_category
0,3,5,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.00,0
1,10,2,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.00,0
2,10,6,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.00,0
3,3,5,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.00,0
4,3,7,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...
512,8,7,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,6.44,1
513,8,7,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,54.29,1
514,8,7,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,11.16,1
515,8,6,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,0.00,0


In [15]:
# Standardizing data

from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

In [16]:
scaler.fit(data.drop('size_category',axis=1))

StandardScaler()

In [17]:
scaled_features=scaler.transform(data.drop('size_category',axis=1))
data_head=pd.DataFrame(scaled_features,columns=data.columns[:-1])
data_head

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,-1.968443,0.357721,-0.805959,-1.323326,-1.830477,-0.860946,-1.842640,0.411724,1.498614,-0.073268,-0.202020
1,1.110120,-1.090909,-0.008102,-1.179541,0.488891,-0.509688,-0.153278,-0.692456,-1.741756,-0.073268,-0.202020
2,1.110120,0.840597,-0.008102,-1.049822,0.560715,-0.509688,-0.739383,-0.692456,-1.518282,-0.073268,-0.202020
3,-1.968443,0.357721,0.191362,-1.212361,-1.898266,-0.004756,-1.825402,3.233519,-0.009834,0.603155,-0.202020
4,-1.968443,1.323474,-0.243833,-0.931043,-1.798600,0.126966,-1.291012,3.356206,-1.238940,-0.073268,-0.202020
...,...,...,...,...,...,...,...,...,...,...,...
512,0.230531,1.323474,-1.640083,-0.846648,0.474768,-1.563460,1.536084,-0.753800,-0.736124,-0.073268,-0.100753
513,0.230531,1.323474,-1.640083,-0.846648,0.474768,-1.563460,0.519019,1.638592,0.995798,-0.073268,0.651674
514,0.230531,1.323474,-1.640083,-0.846648,0.474768,-1.563460,0.398350,1.577248,1.498614,-0.073268,-0.026532
515,0.230531,0.840597,0.680957,0.549003,0.269382,0.500176,1.156839,-0.140366,-0.009834,-0.073268,-0.202020


In [18]:
# Splitting data into test data and train data

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(data_head,data['size_category'], test_size=0.3, random_state=42)

In [19]:
print('Shape of x_train: ', x_train.shape)
print('Shape of x_test: ', x_test.shape)
print('Shape of y_train: ', y_train.shape)
print('Shape of y_test: ', y_test.shape)

Shape of x_train:  (361, 11)
Shape of x_test:  (156, 11)
Shape of y_train:  (361,)
Shape of y_test:  (156,)


In [20]:
# create model
model = Sequential()
model.add(Dense(12, input_dim=11, kernel_initializer='uniform', activation='relu'))
model.add(Dense(10, kernel_initializer='uniform', activation='relu'))
model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))

In [21]:
#compile model
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [22]:
model_fit= model.fit(x_train,y_train,validation_split=0.33,epochs=250,batch_size=10)

Epoch 1/250
25/25 [==============================] - 1s 15ms/step - loss: 0.6905 - accuracy: 0.6846 - val_loss: 0.6863 - val_accuracy: 0.7417
Epoch 2/250
25/25 [==============================] - 0s 4ms/step - loss: 0.6828 - accuracy: 0.7220 - val_loss: 0.6760 - val_accuracy: 0.7417
Epoch 3/250
25/25 [==============================] - 0s 4ms/step - loss: 0.6716 - accuracy: 0.7220 - val_loss: 0.6620 - val_accuracy: 0.7417
Epoch 4/250
25/25 [==============================] - 0s 4ms/step - loss: 0.6522 - accuracy: 0.7220 - val_loss: 0.6369 - val_accuracy: 0.7417
Epoch 5/250
25/25 [==============================] - 0s 5ms/step - loss: 0.6257 - accuracy: 0.7220 - val_loss: 0.6107 - val_accuracy: 0.7417
Epoch 6/250
25/25 [==============================] - 0s 5ms/step - loss: 0.5941 - accuracy: 0.7220 - val_loss: 0.5805 - val_accuracy: 0.7500
Epoch 7/250
25/25 [==============================] - 0s 4ms/step - loss: 0.5660 - accuracy: 0.7220 - val_loss: 0.5574 - val_accuracy: 0.7500
Epoch 8/250


Epoch 59/250
25/25 [==============================] - 0s 4ms/step - loss: 0.0689 - accuracy: 0.9876 - val_loss: 0.0803 - val_accuracy: 0.9750
Epoch 60/250
25/25 [==============================] - 0s 5ms/step - loss: 0.0682 - accuracy: 0.9834 - val_loss: 0.0783 - val_accuracy: 0.9750
Epoch 61/250
25/25 [==============================] - 0s 4ms/step - loss: 0.0650 - accuracy: 0.9876 - val_loss: 0.0768 - val_accuracy: 0.9667
Epoch 62/250
25/25 [==============================] - 0s 5ms/step - loss: 0.0655 - accuracy: 0.9834 - val_loss: 0.0770 - val_accuracy: 0.9667
Epoch 63/250
25/25 [==============================] - 0s 4ms/step - loss: 0.0638 - accuracy: 0.9876 - val_loss: 0.0713 - val_accuracy: 0.9750
Epoch 64/250
25/25 [==============================] - 0s 5ms/step - loss: 0.0588 - accuracy: 0.9876 - val_loss: 0.0695 - val_accuracy: 0.9750
Epoch 65/250
25/25 [==============================] - 0s 4ms/step - loss: 0.0568 - accuracy: 0.9876 - val_loss: 0.0678 - val_accuracy: 0.9833
Epoch 

25/25 [==============================] - 0s 5ms/step - loss: 0.0278 - accuracy: 0.9959 - val_loss: 0.0371 - val_accuracy: 0.9833
Epoch 117/250
25/25 [==============================] - 0s 4ms/step - loss: 0.0281 - accuracy: 0.9876 - val_loss: 0.0354 - val_accuracy: 0.9833
Epoch 118/250
25/25 [==============================] - 0s 5ms/step - loss: 0.0252 - accuracy: 0.9959 - val_loss: 0.0360 - val_accuracy: 0.9750
Epoch 119/250
25/25 [==============================] - 0s 5ms/step - loss: 0.0265 - accuracy: 0.9959 - val_loss: 0.0353 - val_accuracy: 0.9833
Epoch 120/250
25/25 [==============================] - 0s 5ms/step - loss: 0.0249 - accuracy: 0.9959 - val_loss: 0.0352 - val_accuracy: 0.9833
Epoch 121/250
25/25 [==============================] - 0s 5ms/step - loss: 0.0245 - accuracy: 0.9959 - val_loss: 0.0350 - val_accuracy: 0.9750
Epoch 122/250
25/25 [==============================] - 0s 5ms/step - loss: 0.0241 - accuracy: 0.9959 - val_loss: 0.0341 - val_accuracy: 0.9833
Epoch 123/250

25/25 [==============================] - 0s 4ms/step - loss: 0.0200 - accuracy: 0.9959 - val_loss: 0.0269 - val_accuracy: 0.9833
Epoch 174/250
25/25 [==============================] - 0s 4ms/step - loss: 0.0156 - accuracy: 0.9959 - val_loss: 0.0274 - val_accuracy: 0.9833
Epoch 175/250
25/25 [==============================] - 0s 4ms/step - loss: 0.0188 - accuracy: 0.9959 - val_loss: 0.0281 - val_accuracy: 0.9917
Epoch 176/250
25/25 [==============================] - 0s 4ms/step - loss: 0.0186 - accuracy: 0.9917 - val_loss: 0.0267 - val_accuracy: 0.9833
Epoch 177/250
25/25 [==============================] - 0s 4ms/step - loss: 0.0192 - accuracy: 0.9959 - val_loss: 0.0264 - val_accuracy: 0.9833
Epoch 178/250
25/25 [==============================] - 0s 5ms/step - loss: 0.0167 - accuracy: 0.9959 - val_loss: 0.0269 - val_accuracy: 0.9833
Epoch 179/250
25/25 [==============================] - 0s 5ms/step - loss: 0.0163 - accuracy: 0.9959 - val_loss: 0.0276 - val_accuracy: 0.9917
Epoch 180/250

25/25 [==============================] - 0s 5ms/step - loss: 0.0127 - accuracy: 0.9959 - val_loss: 0.0254 - val_accuracy: 0.9833
Epoch 231/250
25/25 [==============================] - 0s 5ms/step - loss: 0.0132 - accuracy: 0.9959 - val_loss: 0.0262 - val_accuracy: 0.9833
Epoch 232/250
25/25 [==============================] - 0s 4ms/step - loss: 0.0151 - accuracy: 0.9959 - val_loss: 0.0305 - val_accuracy: 0.9833
Epoch 233/250
25/25 [==============================] - 0s 5ms/step - loss: 0.0149 - accuracy: 0.9959 - val_loss: 0.0283 - val_accuracy: 0.9833
Epoch 234/250
25/25 [==============================] - 0s 5ms/step - loss: 0.0147 - accuracy: 0.9917 - val_loss: 0.0263 - val_accuracy: 0.9833
Epoch 235/250
25/25 [==============================] - 0s 4ms/step - loss: 0.0139 - accuracy: 0.9959 - val_loss: 0.0272 - val_accuracy: 0.9833
Epoch 236/250
25/25 [==============================] - 0s 5ms/step - loss: 0.0164 - accuracy: 0.9959 - val_loss: 0.0248 - val_accuracy: 0.9833
Epoch 237/250

In [24]:
# evaluate the model
scores = model.evaluate(x_test, y_test)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

5/5 [==============================] - 0s 4ms/step - loss: 0.0427 - accuracy: 0.9744
accuracy: 97.44%
